In [ ]:
import sys, os, inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0, parentdir)

In [1]:
from src.extract import extract
from src.export import export, export_dir, display_progress

from slippi import Game

from pymongo import MongoClient
from bson.binary import Binary
from scipy import sparse
import numpy as np
import pickle
import os

In [2]:
database_name = 'slippi'
collection_name = 'Blynde_clips'
game_id = 'Game_20200712T203020.slp'

# Connect to the hosted MongoDB instance
client = MongoClient('localhost', 27017)
db = client[database_name]
collection = db[collection_name]

In [3]:
player = collection.find_one()

In [6]:
player['_id']

ObjectId('5fc966a6f793665e16832de1')

In [4]:
# retreive player data on single game from database
p1, p2 = tuple(collection.find({'game_id':game_id}))[:2]

In [5]:
# convert istream from "sparse matrix bytestream" to numpy array
print(type(p1['istream']))
if isinstance(p1['istream'], bytes):
    p1['istream'] = pickle.loads( p1['istream']).toarray()
    p2['istream'] = pickle.loads( p2['istream']).toarray()
print(type(p1['istream']))

<class 'bytes'>
<class 'numpy.ndarray'>


In [6]:
print(f"{p1['name']} ({p1['code']}): {p1['character']}")
print(f"{p2['name']} ({p2['code']}): {p2['character']}")

Spriggs (RYAN#558): FALCO
Blynde (BLYN#929): ICE_CLIMBERS


In [7]:
def presses(buttonstream):
    return int(np.sum(np.abs(np.diff(buttonstream, axis=0)))/2)

In [8]:
resultA = presses(p1['istream'][:, 9]) > presses(p2['istream'][:, 9])
resultB = presses(p1['istream'][:, 8]) > presses(p2['istream'][:, 8])
resultXY = presses(p1['istream'][:, [6,7]]) > presses(p2['istream'][:, [6,7]])
resultLR = presses(p1['istream'][:, [10,11]]) > presses(p2['istream'][:, [10,11]])
resultZ = presses(p1['istream'][:, 12]) > presses(p2['istream'][:, 12])

whoA = p1 if resultA else p2
whoB = p1 if resultB else p2
whoXY = p1 if resultXY else p2
whoLR = p1 if resultLR else p2
whoZ = p1 if resultZ else p2

print(f"{p1['name']}: {p1['character']}")
print('---VS---')
print(f"{p2['name']}: {p2['character']}")

print(f"\nMatch Length: {round(len(p1['istream'])/3600, 2)} minutes")
print(f"Match ID: {p1['game_id']}")

print(f"\nMost A presses: {presses(whoA['istream'][:, 9])}\n --- {whoA['name']} - {whoA['character']}")
print(f"\nMost B presses: {presses(whoB['istream'][:, 8])}\n --- {whoB['name']} - {whoB['character']}")
print(f"\nMost X or Y presses: {presses(whoXY['istream'][:, [6,7]])}\n --- {whoXY['name']} - {whoXY['character']}")
print(f"\nMost L or R presses: {presses(whoLR['istream'][:, [10,11]])}\n --- {whoLR['name']} - {whoLR['character']}")
print(f"\nMost Z presses: {presses(whoZ['istream'][:, 12])}\n --- {whoZ['name']} - {whoZ['character']}")

Spriggs: FALCO
---VS---
Blynde: ICE_CLIMBERS

Match Length: 4.23 minutes
Match ID: Game_20200712T203020.slp

Most A presses: 178
 --- Blynde - ICE_CLIMBERS

Most B presses: 100
 --- Blynde - ICE_CLIMBERS

Most X or Y presses: 174
 --- Spriggs - FALCO

Most L or R presses: 106
 --- Blynde - ICE_CLIMBERS

Most Z presses: 15
 --- Blynde - ICE_CLIMBERS


In [9]:
if presses(p1['istream'][:, 6]) > presses(p1['istream'][:, 7]):
    jump_button = 'Y'
else:
    jump_button = 'X'
print(f"{p1['name']} uses {jump_button} to jump")

if presses(p2['istream'][:, 6]) > presses(p2['istream'][:, 7]):
    jump_button = 'Y'
else:
    jump_button = 'X'
print(f"{p2['name']} uses {jump_button} to jump")

Spriggs uses Y to jump
Blynde uses Y to jump


In [23]:
database_name = 'slippi'
collection_name = 'gh0st'

# Connect to the hosted MongoDB instance
client = MongoClient('localhost', 27017)
db = client[database_name]
collection = db[collection_name]

In [24]:
collection.count_documents({'code' : 'GOOS#845'})

0

In [25]:
collection.count_documents({'code' : 'GHST#265'})

283

In [26]:
collection.count_documents({'code' : 'CUCK#434'})

0